In [1]:
import pandas as pd
import seaborn as sns
from merf.merf import MERF
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt
import numpy as np



def calculate_metrics(y_true, y_pred):    
    r2 = r2_score(y_true, y_pred)
    print(" R2 of the model is ", r2)
    rms = sqrt(mean_squared_error(y_true, y_pred))
    print(" RMSE of the model is ", rms)
    mae = mean_absolute_error(y_true, y_pred)
    print(" MAE of the model is ", mae)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(" MAPE of the model is ", mape)
    
def make_plot(y_test, y_pred):
    plt.figure(figsize=(15, 5))
    plt.plot(y_pred)
    plt.plot(y_test.values)
    plt.legend(['Predicted', 'Actual'])
    
def make_scatterplot(y_test, y_pred):
    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')

def split_by_time(data, time):
    train = data[(data['time'] < time)]
    test = data[(data['time'] >= time)]
    X_train = train[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]
    y_train = train[['head']]
    X_test = test[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]
    y_test = test[['head']]
    return X_train, y_train, X_test, y_test

def test_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    calculate_metrics(y_test, y_pred)
    make_plot(y_test, y_pred)

In [2]:
de = pd.read_csv('de-clean.csv')
de['site'] = 1
nl = pd.read_csv('nl-clean.csv')
nl['site'] = 2
se1 = pd.read_csv('se1-clean.csv')
se1['site'] = 3
se2 = pd.read_csv('se2-clean.csv')
se2['site'] = 4

In [3]:
len(se2.columns)

12

In [4]:
de

,time,head,rr,tg,tn,tx,pp,hu,fg,qq,et,site
0,2002-05-01,374.76,0.0,12.990000,7.42,17.630000,1008.9000,70.520004,2.88,205.0,2.765489,1
1,2002-05-02,374.75,0.0,11.759999,7.08,16.109999,1008.0000,79.690000,2.70,154.0,2.015525,1
2,2002-05-03,374.74,0.0,11.880000,8.66,16.779999,1007.7000,89.940000,2.49,102.0,1.339214,1
3,2002-05-04,374.72,6.3,8.670000,5.98,10.950000,1009.5000,94.128580,2.40,85.0,1.022122,1
4,2002-05-05,374.71,7.0,7.870000,3.90,13.219999,1015.2000,77.665000,1.87,137.0,1.604829,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5354,2016-12-27,374.55,0.0,2.470000,1.15,3.530000,1039.9000,84.810000,4.71,27.0,0.260880,1
5355,2016-12-28,374.55,0.0,3.510000,0.50,4.260000,1043.1000,85.750000,3.62,28.0,0.280443,1
5356,2016-12-29,374.55,0.0,-1.310000,-4.30,4.440000,1042.1000,95.075005,2.09,51.0,0.424098,1
5357,2016-12-30,374.55,0.0,-4.550000,-8.41,0.890000,1040.6000,94.700005,1.57,50.0,0.360732,1


In [5]:
data = de.append([nl, se1, se2])

/tmp/ipykernel_423869/301329737.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = de.append([nl, se1, se2])


In [6]:
data

,time,head,rr,tg,tn,tx,pp,hu,fg,qq,et,site
0,2002-05-01,374.76,0.0,12.990000,7.420000,17.630000,1008.9,70.520004,2.88,205.0,2.765489,1
1,2002-05-02,374.75,0.0,11.759999,7.080000,16.109999,1008.0,79.690000,2.70,154.0,2.015525,1
2,2002-05-03,374.74,0.0,11.880000,8.660000,16.779999,1007.7,89.940000,2.49,102.0,1.339214,1
3,2002-05-04,374.72,6.3,8.670000,5.980000,10.950000,1009.5,94.128580,2.40,85.0,1.022122,1
4,2002-05-05,374.71,7.0,7.870000,3.900000,13.219999,1015.2,77.665000,1.87,137.0,1.604829,1
...,...,...,...,...,...,...,...,...,...,...,...,...
778,2015-12-01,348.12,1.4,-3.090000,-6.560000,-1.350000,990.9,86.820000,1.30,1.0,0.007958,4
779,2015-12-08,348.05,0.0,-2.610000,-5.390000,0.370000,1013.4,76.980000,2.40,1.0,0.008009,4
780,2015-12-15,347.96,0.0,-5.730000,-7.520000,-3.030000,1010.3,82.415000,2.83,1.0,0.006965,4
781,2015-12-22,347.88,0.0,-2.490000,-5.170000,0.290000,988.7,87.620000,1.17,1.0,0.008178,4


In [7]:
data = data.drop('time', axis=1)

In [8]:
data

,head,rr,tg,tn,tx,pp,hu,fg,qq,et,site
0,374.76,0.0,12.990000,7.420000,17.630000,1008.9,70.520004,2.88,205.0,2.765489,1
1,374.75,0.0,11.759999,7.080000,16.109999,1008.0,79.690000,2.70,154.0,2.015525,1
2,374.74,0.0,11.880000,8.660000,16.779999,1007.7,89.940000,2.49,102.0,1.339214,1
3,374.72,6.3,8.670000,5.980000,10.950000,1009.5,94.128580,2.40,85.0,1.022122,1
4,374.71,7.0,7.870000,3.900000,13.219999,1015.2,77.665000,1.87,137.0,1.604829,1
...,...,...,...,...,...,...,...,...,...,...,...
778,348.12,1.4,-3.090000,-6.560000,-1.350000,990.9,86.820000,1.30,1.0,0.007958,4
779,348.05,0.0,-2.610000,-5.390000,0.370000,1013.4,76.980000,2.40,1.0,0.008009,4
780,347.96,0.0,-5.730000,-7.520000,-3.030000,1010.3,82.415000,2.83,1.0,0.006965,4
781,347.88,0.0,-2.490000,-5.170000,0.290000,988.7,87.620000,1.17,1.0,0.008178,4


In [11]:
#train, test = train_test_split(data, test_size=0, shuffle=True)

In [10]:
y = data['head']

In [12]:
mrf = MERF(max_iterations=10)

In [13]:
data.columns

Index(['head', 'rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et', 'site'], dtype='object')

In [14]:
X_train =  data[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]

In [15]:
Z_train = np.ones((len(X_train), 1))

In [17]:
clusters_train = data['site']

In [18]:
y_train = data['head']

In [19]:
mrf.fit(X_train, Z_train, clusters_train, y_train)

INFO     [merf.py:307] Training GLL is 99341.19119352459 at iteration 1.
INFO     [merf.py:307] Training GLL is 92821.72972311151 at iteration 2.
INFO     [merf.py:307] Training GLL is 86718.63662644189 at iteration 3.
INFO     [merf.py:307] Training GLL is 80483.65884939874 at iteration 4.
INFO     [merf.py:307] Training GLL is 74416.96339676401 at iteration 5.
INFO     [merf.py:307] Training GLL is 68693.14039840706 at iteration 6.
INFO     [merf.py:307] Training GLL is 63253.62492735704 at iteration 7.
INFO     [merf.py:307] Training GLL is 58069.775637541345 at iteration 8.
INFO     [merf.py:307] Training GLL is 53206.27400850056 at iteration 9.
INFO     [merf.py:307] Training GLL is 48588.1199760861 at iteration 10.


In [ ]:
X_test =  test[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]

## Predict for Germany

In [127]:
x = pd.read_csv('./Germany/input_data.csv')
de_test = x
de_test['site'] = 1

In [128]:
de_test

,time,rr,tg,tn,tx,pp,hu,fg,qq,et,site
0,1990-01-01,0.000000,-4.47,-5.31,-3.860000,1022.40000,92.150000,1.03,30.0,0.219842,1
1,1990-01-02,0.000000,-2.74,-5.45,-0.430000,1023.50000,91.375000,0.99,33.0,0.261125,1
2,1990-01-03,0.000000,-1.05,-3.52,-0.290000,1024.40000,86.180000,2.09,15.0,0.127449,1
3,1990-01-04,0.000000,-3.00,-4.11,-0.790000,1027.90000,88.585000,2.22,30.0,0.234045,1
4,1990-01-05,1.400000,-1.46,-3.50,-0.870000,1027.80000,92.460010,1.56,14.0,0.116699,1
...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,1.100000,1.30,0.72,2.920000,1009.50000,94.450005,2.49,27.0,0.254015,1
11684,2021-12-28,18.300001,3.75,1.42,8.679999,1004.00000,93.175000,3.34,17.0,0.175350,1
11685,2021-12-29,6.600000,7.51,6.62,8.940000,1007.10004,87.680000,4.40,22.0,0.255928,1
11686,2021-12-30,5.100000,10.95,7.19,13.340000,1019.40000,92.815796,3.47,20.0,0.255054,1


In [129]:
test_data = de_test
test_data = test_data.dropna()
Z_test = np.ones((len(test_data), 1))
clusters_test = test_data['site']
X_test =  test_data[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]
yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)

In [130]:
test_data

,time,rr,tg,tn,tx,pp,hu,fg,qq,et,site
0,1990-01-01,0.000000,-4.47,-5.31,-3.860000,1022.40000,92.150000,1.03,30.0,0.219842,1
1,1990-01-02,0.000000,-2.74,-5.45,-0.430000,1023.50000,91.375000,0.99,33.0,0.261125,1
2,1990-01-03,0.000000,-1.05,-3.52,-0.290000,1024.40000,86.180000,2.09,15.0,0.127449,1
3,1990-01-04,0.000000,-3.00,-4.11,-0.790000,1027.90000,88.585000,2.22,30.0,0.234045,1
4,1990-01-05,1.400000,-1.46,-3.50,-0.870000,1027.80000,92.460010,1.56,14.0,0.116699,1
...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,1.100000,1.30,0.72,2.920000,1009.50000,94.450005,2.49,27.0,0.254015,1
11684,2021-12-28,18.300001,3.75,1.42,8.679999,1004.00000,93.175000,3.34,17.0,0.175350,1
11685,2021-12-29,6.600000,7.51,6.62,8.940000,1007.10004,87.680000,4.40,22.0,0.255928,1
11686,2021-12-30,5.100000,10.95,7.19,13.340000,1019.40000,92.815796,3.47,20.0,0.255054,1


In [131]:
len(yhat_mrf)

11688

In [132]:
std = 0.11559471217735248**0.5
upper = yhat_mrf + 1.96*std
lower = yhat_mrf - 1.96*std

In [134]:
de_sub = pd.DataFrame({'Date':test_data['time'], 'Simulated Head':yhat_mrf, '95% Lower Bound':lower, '95% Upper Bound':upper})
de_sub.to_csv('submission_form_Germany.csv', index=False)
de_sub

,Date,Simulated Head,95% Lower Bound,95% Upper Bound
0,1990-01-01,382.237360,381.570975,382.903745
1,1990-01-02,381.171695,380.505310,381.838080
2,1990-01-03,365.869900,365.203515,366.536284
3,1990-01-04,367.012403,366.346018,367.678788
4,1990-01-05,352.183246,351.516861,352.849631
...,...,...,...,...
11683,2021-12-27,371.462091,370.795706,372.128475
11684,2021-12-28,357.101041,356.434656,357.767426
11685,2021-12-29,354.727329,354.060944,355.393713
11686,2021-12-30,347.307221,346.640836,347.973605


## Predict for Netherlands

In [137]:
x = pd.read_csv('./Netherlands/input_data.csv')
nl_test = x
nl_test['site'] = 2

test_data = nl_test
test_data = test_data.dropna()
Z_test = np.ones((len(test_data), 1))
clusters_test = test_data['site']
X_test =  test_data[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]
yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)

In [138]:
std = 0.11559471217735248**0.5
upper = yhat_mrf + 1.96*std
lower = yhat_mrf
nl_sub = pd.DataFrame({'Date':test_data['time'], 'Simulated Head':yhat_mrf, '95% Lower Bound':lower, '95% Upper Bound':upper})
nl_sub.to_csv('submission_form_Netherlands.csv', index=False)
nl_sub

,Date,Simulated Head,95% Lower Bound,95% Upper Bound
0,1990-01-01,72.907151,72.907151,73.573536
1,1990-01-02,12.104081,12.104081,12.770466
2,1990-01-03,10.627332,10.627332,11.293716
3,1990-01-04,15.584060,15.584060,16.250444
4,1990-01-05,18.368441,18.368441,19.034826
...,...,...,...,...
11683,2021-12-27,20.275823,20.275823,20.942208
11684,2021-12-28,15.178053,15.178053,15.844438
11685,2021-12-29,37.607263,37.607263,38.273648
11686,2021-12-30,11.905404,11.905404,12.571789


## Predict for Sweden 1

In [139]:
x = pd.read_csv('./Sweden_1/input_data.csv')
se1_test = x
se1_test['site'] = 3

test_data = se1_test
test_data = test_data.dropna()
Z_test = np.ones((len(test_data), 1))
clusters_test = test_data['site']
X_test =  test_data[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]
yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)

In [140]:
std = 0.11559471217735248**0.5
upper = yhat_mrf + 1.96*std
lower = yhat_mrf
se1_sub = pd.DataFrame({'Date':test_data['time'], 'Simulated Head':yhat_mrf, '95% Lower Bound':lower, '95% Upper Bound':upper})
se1_sub.to_csv('submission_form_Sweden_1.csv', index=False)
se1_sub

,Date,Simulated Head,95% Lower Bound,95% Upper Bound
14,1990-01-15,243.553302,243.553302,244.219687
15,1990-01-16,242.517148,242.517148,243.183533
16,1990-01-17,243.519693,243.519693,244.186077
17,1990-01-18,244.374490,244.374490,245.040875
18,1990-01-19,242.675931,242.675931,243.342316
...,...,...,...,...
11683,2021-12-27,244.220200,244.220200,244.886585
11684,2021-12-28,244.247082,244.247082,244.913467
11685,2021-12-29,244.341310,244.341310,245.007695
11686,2021-12-30,244.787341,244.787341,245.453726


## Predict for Sweden 2

In [142]:
x = pd.read_csv('./Sweden_2/input_data.csv')
se2_test = x
se2_test['site'] = 4

test_data = se2_test
test_data = test_data.dropna()
Z_test = np.ones((len(test_data), 1))
clusters_test = test_data['site']
X_test =  test_data[['rr', 'tg', 'tn', 'tx', 'pp', 'hu', 'fg', 'qq', 'et']]
yhat_mrf = mrf.predict(X_test, Z_test, clusters_test)

In [143]:
std = 0.11559471217735248**0.5
upper = yhat_mrf + 1.96*std
lower = yhat_mrf
se2_sub = pd.DataFrame({'Date':test_data['time'], 'Simulated Head':yhat_mrf, '95% Lower Bound':lower, '95% Upper Bound':upper})
se2_sub.to_csv('submission_form_Sweden_2.csv', index=False)
se2_sub

,Date,Simulated Head,95% Lower Bound,95% Upper Bound
19,1990-01-20,352.099604,352.099604,352.765988
20,1990-01-21,352.413633,352.413633,353.080017
21,1990-01-22,352.027200,352.027200,352.693584
22,1990-01-23,352.419357,352.419357,353.085742
23,1990-01-24,351.925594,351.925594,352.591978
...,...,...,...,...
11683,2021-12-27,351.678471,351.678471,352.344855
11684,2021-12-28,352.091047,352.091047,352.757431
11685,2021-12-29,352.007505,352.007505,352.673890
11686,2021-12-30,352.447949,352.447949,353.114334
